In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from numpy.random import rand
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [2]:
def error_rate(features, target, x, opts):
    # parameters
    k     = opts['k']
    cv    = opts['cv']

    kf = KFold(n_splits=cv, shuffle=True, random_state=2)

    total_error = 0

    for train_index, test_index in kf.split(features):
      X_train, X_test = features[train_index], features[test_index]
      y_train, y_test = target[train_index], target[test_index]

      # Number of instances
      num_train = np.size(X_train, 0)
      num_test  = np.size(X_test, 0)
      
      # Define selected features
      xtrain = X_train[:, x==1]
      ytrain = y_train.reshape(num_train)
      xtest  = X_test[:, x==1]
      ytest  = y_test.reshape(num_test)

      # Training
      knn = KNeighborsClassifier(n_neighbors=k)
      knn.fit(xtrain, ytrain)

      # Prediction
      ypred = knn.predict(xtest)
      acc   = np.sum(ytest == ypred) / num_test
      error = 1 - acc

      total_error = total_error + error
      
    return total_error/cv

In [3]:
def Fun(features, target, x, opts):
    # Parameters
    alpha    = 0.99
    beta     = 1 - alpha
    # Original feature size
    max_feat = len(x)
    # Number of selected features
    num_feat = np.sum(x == 1)
    # Solve if no feature selected
    if num_feat == 0:
        cost  = 1
    else:
        # Get error rate
        error = error_rate(features, target, x, opts)
        # Objective function
        cost  = alpha * error + beta * (num_feat / max_feat)
        
    return cost

In [4]:
def init_position(lb, ub, N, dim):
    X = np.zeros([N, dim], dtype='float')
    R = rand()
    u = 1 + R
    for d in range(dim):
        X[0,d] = lb[0,d] + (ub[0,d] - lb[0,d]) * rand()
    for i in range(1, N):
        for d in range(dim):
            if X[i-1,d] <= 0.5:
                X[i,d] = u * X[i-1,d]
            else:
                X[i,d] = u * (1 - X[i-1,d])
    
    return X

In [5]:
def binary_conversion(X, thres, N, dim):
    Xbin = np.zeros([N, dim], dtype='int')
    for i in range(N):
        for d in range(dim):
            xi = 1/(1 + np.exp(-X[i,d]))
            if xi > rand():
            # if X[i,d] > 0.5:
                Xbin[i,d] = 0
            else:
                Xbin[i,d] = 1
    
    return Xbin

In [6]:
def boundary(x, lb, ub):
    if x < lb:
        x = lb
    if x > ub:
        x = ub
    
    return x

In [7]:
def woa(features, target, opts):
    # Parameters
    ub    = 1
    lb    = 0
    thres = rand()
    b     = 1       # constant
    
    N        = opts['N']
    max_iter = opts['T']
    if 'b' in opts:
        b    = opts['b']
    
    # Dimension
    dim = np.size(features, 1)
    if np.size(lb) == 1:
        ub = ub * np.ones([1, dim], dtype='float')
        lb = lb * np.ones([1, dim], dtype='float')
        
    # Initialize position 
    X    = init_position(lb, ub, N, dim)
    
    # Binary conversion
    Xbin = binary_conversion(X, thres, N, dim)
    
    # Fitness at first iteration
    fit  = np.zeros([N, 1], dtype='float')
    Xgb  = np.zeros([1, dim], dtype='float')
    fitG = float('inf')
    
    for i in range(N):
        fit[i,0] = Fun(features, target, Xbin[i,:], opts)
        if fit[i,0] < fitG:
            Xgb[0,:] = X[i,:]
            fitG     = fit[i,0]
        
    # Pre
    curve = np.zeros([1, max_iter], dtype='float') 
    t     = 0
    
    curve[0,t] = fitG.copy()
    print("Generation:", t + 1)
    print("Best (WOA):", curve[0,t])
    t += 1

    while t < max_iter:
        # Define a, linearly decreases from 2 to 0 
        a = 2 - t * (2 / max_iter)
        
        for i in range(N):
            # Parameter A (2.3)
            A = 2 * a * rand() - a
            # Paramater C (2.4)
            C = 2 * rand()
            # Parameter p, random number in [0,1]
            p = rand()
            # Parameter l, random number in [-1,1]
            l = -1 + 2 * rand()  
            # Whale position update (2.6)
            if p  < 0.5:
                # {1} Encircling prey
                if abs(A) < 1:
                    for d in range(dim):
                        # Compute D (2.1)
                        Dx     = abs(C * Xgb[0,d] - X[i,d])
                        # Position update (2.2)
                        X[i,d] = Xgb[0,d] - A * Dx
                        # Boundary
                        X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])
                
                # {2} Search for prey
                elif abs(A) >= 1:
                    for d in range(dim):
                        # Select a random whale
                        k      = np.random.randint(low = 0, high = N)
                        # Compute D (2.7)
                        Dx     = abs(C * X[k,d] - X[i,d])
                        # Position update (2.8)
                        X[i,d] = X[k,d] - A * Dx
                        # Boundary
                        X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])
            
            # {3} Bubble-net attacking 
            elif p >= 0.5:
                for d in range(dim):
                    # Distance of whale to prey
                    dist   = abs(Xgb[0,d] - X[i,d])
                    # Position update (2.5)
                    X[i,d] = dist * np.exp(b * l) * np.cos(2 * np.pi * l) + Xgb[0,d] 
                    # Boundary
                    X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])
        
        # Binary conversion
        Xbin = binary_conversion(X, thres, N, dim)
        
        # Fitness
        for i in range(N):
            fit[i,0] = Fun(features, target, Xbin[i,:], opts)
            if fit[i,0] < fitG:
                Xgb[0,:] = X[i,:]
                fitG     = fit[i,0]
        
        # Store result
        curve[0,t] = fitG.copy()
        print("Generation:", t + 1)
        print("Best (WOA):", curve[0,t])
        t += 1            

            
    # Best feature subset
    Gbin       = binary_conversion(Xgb, thres, 1, dim) 
    Gbin       = Gbin.reshape(dim)    
    pos        = np.asarray(range(0, dim))    
    sel_index  = pos[Gbin == 1]
    num_feat   = len(sel_index)
    # Create dictionary
    woa_data = {'sf': sel_index, 'c': curve, 'nf': num_feat}
    
    return woa_data 

In [8]:
column_headers = ['Type', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols',
                  'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color Intensity',
                   'Hue', 'OD280/OD315 of diluted wines', 'Proline']

In [9]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None, names=column_headers)
data = data.values
data

array([[1.000e+00, 1.423e+01, 1.710e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.000e+00, 1.320e+01, 1.780e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.000e+00, 1.316e+01, 2.360e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [3.000e+00, 1.327e+01, 4.280e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [3.000e+00, 1.317e+01, 2.590e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [3.000e+00, 1.413e+01, 4.100e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [10]:
features  = np.asarray(data[:, 1:])
target = np.asarray(data[:, 0])

In [11]:
num_runs = 20    # number of independent runs
k        = 5     # k-value in KNN
N        = 5     # number of particles
T        = 20    # maximum number of iterations
cv       = 10    # K-fold cross-validation
opts     = {'k':k, 'N':N, 'T':T, 'cv': cv}
dim      = features.shape[1]

In [12]:
acc_arr          = []
feature_size_arr = []
run_count        = 0

while run_count < num_runs:
  run_count += 1
  print("Run ", run_count)
  print("-----------------------------------")
  fmdl = woa(features, target, opts)
  sf   = fmdl['sf']
  
  if sf.size == 0:
    sf = np.arange(dim)
    
  print("SF", sf)

  kf = KFold(n_splits=cv, shuffle=True, random_state=2)

  total_Acc = 0

  for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = target[train_index], target[test_index]

    # Number of instances
    num_train = np.size(X_train, 0)
    num_test  = np.size(X_test, 0)
    
    # Define selected features
    xtrain = X_train[:, sf]
    ytrain = y_train.reshape(num_train)
    xtest  = X_test[:, sf]
    ytest  = y_test.reshape(num_test)

    # Training
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(xtrain, ytrain)

    # Prediction
    ypred = knn.predict(xtest)
    acc   = np.sum(ytest == ypred) / num_test

    total_Acc = total_Acc + acc

  Accuracy = 100 * (total_Acc/cv)
  print("Accuracy:", Accuracy)
  acc_arr.append(Accuracy)

  num_feat = fmdl['nf']
  print("Feature Size:", num_feat)
  feature_size_arr.append(num_feat)
  print("--------------------------------------------------")

Run  1
-----------------------------------
Generation: 1
Best (WOA): 0.08699321266968328
Generation: 2
Best (WOA): 0.08699321266968328
Generation: 3
Best (WOA): 0.08699321266968328
Generation: 4
Best (WOA): 0.08699321266968328
Generation: 5
Best (WOA): 0.06543891402714935
Generation: 6
Best (WOA): 0.06543891402714935
Generation: 7
Best (WOA): 0.06543891402714935
Generation: 8
Best (WOA): 0.06543891402714935
Generation: 9
Best (WOA): 0.06543891402714935
Generation: 10
Best (WOA): 0.06543891402714935
Generation: 11
Best (WOA): 0.06543891402714935
Generation: 12
Best (WOA): 0.06543891402714935
Generation: 13
Best (WOA): 0.06543891402714935
Generation: 14
Best (WOA): 0.06026244343891405
Generation: 15
Best (WOA): 0.06026244343891405
Generation: 16
Best (WOA): 0.06026244343891405
Generation: 17
Best (WOA): 0.06026244343891405
Generation: 18
Best (WOA): 0.06026244343891405
Generation: 19
Best (WOA): 0.06026244343891405
Generation: 20
Best (WOA): 0.06026244343891405
SF [ 0  2  4  7  9 10 11]


In [13]:
print("Average Accuracy: ", np.mean(acc_arr))
print("Average number of features selected: ", np.mean(feature_size_arr))

Average Accuracy:  75.0032679738562
Average number of features selected:  4.6
